In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import clear_output

import quick_pp.las_handler as las
from quick_pp.lithology.multi_mineral import MultiMineral
from quick_pp.plotter.plotter import plotly_log
from quick_pp.qaqc import badhole_flagging, handle_outer_limit

In [ ]:
from collections import OrderedDict
from quick_pp.plotter.well_log_config import COLOR_DICT

TRACE_DEFS = OrderedDict(
    GR=dict(
        track=1,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_width': 1, 'line_color': COLOR_DICT['GR']}
    ),
    RT=dict(
        track=2,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_dash': 'dot', 'line_width': 1, 'line_color': COLOR_DICT['RT']}
    ),
    RHOB=dict(
        track=3,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_width': 1, 'line_color': COLOR_DICT['RHOB']}
    ),
    NPHI=dict(
        track=4,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_width': 1, 'line_color': COLOR_DICT['NPHI']}
    ),
    PEF=dict(
        track=5,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_width': .75, 'line_color': COLOR_DICT['PEF']}
    ),
    DTC=dict(
        track=6,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_width': 1, 'line_color': COLOR_DICT['DTC']}
    ),
    PHIT=dict(
        track=7,
        secondary_y=True,
        hide_xaxis=False,
        style={'line_width': 1, 'line_color': 'blue'}
    ),
    VCLAY=dict(
        track=8,
        secondary_y=False,
        hide_xaxis=False,
        style={
            'line_width': .5, 'line_color': 'black', 'fill': 'tozerox', 'fillpattern_bgcolor': COLOR_DICT['VCLAY'],
            'fillpattern_fgcolor': '#000000', 'fillpattern_fillmode': 'replace', 'fillpattern_shape': '-',
            'fillpattern_size': 2, 'fillpattern_solidity': 0.1, 'stackgroup': 'litho', 'orientation': 'h'
        }
    ),
    AVG_ERROR=dict(
        track=9,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_width': 1, 'line_color': 'black'}
    ),
    VSILT=dict(
        track=8,
        secondary_y=False,
        hide_xaxis=True,
        style={
            'line_width': .5, 'line_color': 'black', 'fill': 'tonextx', 'fillpattern_bgcolor': COLOR_DICT['VSILT'],
            'fillpattern_fgcolor': '#000000', 'fillpattern_fillmode': 'replace', 'fillpattern_shape': '.',
            'fillpattern_size': 3, 'fillpattern_solidity': 0.1, 'stackgroup': 'litho', 'orientation': 'h'
        }
    ),
    VSAND=dict(
        track=8,
        secondary_y=False,
        hide_xaxis=True,
        style={
            'line_width': .5, 'line_color': 'black', 'fill': 'tonextx', 'fillpattern_bgcolor': COLOR_DICT['VSAND'],
            'fillpattern_fgcolor': '#000000', 'fillpattern_fillmode': 'replace', 'fillpattern_shape': '.',
            'fillpattern_size': 3, 'fillpattern_solidity': 0.1, 'stackgroup': 'litho', 'orientation': 'h'
        }
    ),
    VCALC=dict(
        track=8,
        secondary_y=False,
        hide_xaxis=True,
        style={
            'line_width': .5, 'line_color': 'black', 'fill': 'tonextx', 'fillpattern_bgcolor': COLOR_DICT['VCALC'],
            'fillpattern_fgcolor': '#000000', 'fillpattern_fillmode': 'replace', 'fillpattern_shape': '.',
            'fillpattern_size': 3, 'fillpattern_solidity': 0.1, 'stackgroup': 'litho', 'orientation': 'h'
        }
    ),
    VDOLO=dict(
        track=8,
        secondary_y=False,
        hide_xaxis=True,
        style={
            'line_width': .5, 'line_color': 'black', 'fill': 'tonextx', 'fillpattern_bgcolor': COLOR_DICT['VDOLO'],
            'fillpattern_fgcolor': '#000000', 'fillpattern_fillmode': 'replace', 'fillpattern_shape': '-',
            'fillpattern_size': 3, 'fillpattern_solidity': 0.3, 'stackgroup': 'litho', 'orientation': 'h'
        }
    ),
    VANHY=dict(
        track=8,
        secondary_y=False,
        hide_xaxis=True,
        style={
            'line_width': .5, 'line_color': 'black', 'fill': 'tonextx', 'fillpattern_bgcolor': COLOR_DICT['VANHY'],
            'fillpattern_fgcolor': '#000000', 'fillpattern_fillmode': 'replace', 'fillpattern_shape': 'x',
            'fillpattern_size': 4, 'fillpattern_solidity': 0.1, 'stackgroup': 'litho', 'orientation': 'h'
        }
    ),
    VGAS=dict(
        track=8,
        secondary_y=False,
        hide_xaxis=True,
        style={
            'line_width': .1, 'line_color': 'black', 'fill': 'tonextx', 'fillcolor': COLOR_DICT['VGAS'],
            'stackgroup': 'litho', 'orientation': 'h'
        }
    ),
    VOIL=dict(
        track=8,
        secondary_y=False,
        hide_xaxis=True,
        style={
            'line_width': .1, 'line_color': 'black', 'fill': 'tonextx', 'fillcolor': COLOR_DICT['VOIL'],
            'stackgroup': 'litho', 'orientation': 'h'
        }
    ),
    VWATER=dict(
        track=8,
        secondary_y=False,
        hide_xaxis=True,
        style={
            'line_width': .1, 'line_color': 'black', 'fill': 'tonextx', 'fillcolor': COLOR_DICT['VWATER'],
            'stackgroup': 'litho', 'orientation': 'h'
        }
    ),
    GR_RECONSTRUCTED=dict(
        track=1,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_dash': 'dot', 'line_width': 1, 'line_color': 'black'}
    ),
    RHOB_RECONSTRUCTED=dict(
        track=3,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_dash': 'dot', 'line_width': 1, 'line_color': 'black'}
    ),
    NPHI_RECONSTRUCTED=dict(
        track=4,
        secondary_y=True,
        hide_xaxis=False,
        style={'line_dash': 'dot', 'line_width': 1, 'line_color': 'black'}
    ),
    PEF_RECONSTRUCTED=dict(
        track=5,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_dash': 'dot', 'line_width': 1, 'line_color': 'black'}
    ),
    DTC_RECONSTRUCTED=dict(
        track=6,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_dash': 'dot', 'line_width': 1, 'line_color': 'black'}
    ),
    PHIT_CONSTRUCTED=dict(
        track=7,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_dash': 'dot', 'line_width': 1, 'line_color': 'black'}
    ),
    CALI=dict(
        track=1,
        secondary_y=False,
        hide_xaxis=False,
        style={
            'line_dash': 'dot', 'line_width': 1, 'line_color': COLOR_DICT['CALI'], 'fill': 'tozerox',
            'fillcolor': 'rgba(165, 42, 42, .15)'
        }
    ),
)

In [ ]:
font_size = 8
XAXIS_DEFS = {
    'GR': {
        'title': {'text': 'GR', 'font': {'color': COLOR_DICT['GR'], 'size': font_size}},
        'tickfont': {'color': COLOR_DICT['GR'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .85,
        'title_standoff': .1, 'dtick': 40, 'range': [0, 200], 'type': 'linear', 'zeroline': False
    },
    'RT': {
        'title': {'text': 'RT', 'font': {'color': COLOR_DICT['RT'], 'size': font_size}},
        'tickfont': {'color': COLOR_DICT['RT'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .85,
        'title_standoff': .1, 'range': [np.log10(.2), np.log10(2000)], 'type': 'log',
        'tickmode': 'array', 'tickvals': np.geomspace(0.2, 2000, 5), 'tickangle': -90, 'minor_showgrid': True
    },
    'RHOB': {
        'title': {'text': 'RHOB', 'font': {'color': COLOR_DICT['RHOB'], 'size': font_size}},
        'tickformat': ".2f", 'tick0': 1.95, 'dtick': 0.2, 'tickangle': -90,
        'tickfont': {'color': COLOR_DICT['RHOB'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .85,
        'title_standoff': .1, 'range': [1.95, 2.95], 'type': 'linear'
    },
    'NPHI': {
        'title': {'text': 'NPHI', 'font': {'color': COLOR_DICT['NPHI'], 'size': font_size}},
        'tickfont': {'color': COLOR_DICT['NPHI'], 'size': font_size}, 'zeroline': False,
        'side': 'top', 'anchor': 'free', 'position': .85, 'title_standoff': .1,
        'tickformat': ".2f", 'tick0': -.15, 'dtick': 0.12, 'range': [.45, -.15], 'type': 'linear', 'tickangle': -90
    },
    'VCLAY': {
        'title': {'text': 'LITHOLOGY', 'font': {'color': COLOR_DICT['VSHALE'], 'size': font_size}},
        'tickfont': {'color': COLOR_DICT['VSHALE'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .85, 'title_standoff': .1, 'type': 'linear', 'zeroline': False
    },
    'DTC': {
        'title': {'text': 'DTC', 'font': {'color': COLOR_DICT['DTC'], 'size': font_size}},
        'tickfont': {'color': COLOR_DICT['DTC'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .85,
        'title_standoff': .1, 'dtick': 20, 'range': [30, 150], 'type': 'linear', 'zeroline': False
    },
    'PEF': {
        'title': {'text': 'PEF', 'font': {'color': COLOR_DICT['PEF'], 'size': font_size}},
        'tickfont': {'color': COLOR_DICT['PEF'], 'size': font_size}, 'zeroline': False,
        'side': 'top', 'anchor': 'free', 'position': .85, 'title_standoff': .1,
        'dtick': 1, 'range': [0, 7], 'type': 'linear', 'showgrid': False
    },
    'GR_RECONSTRUCTED': {
        'title': {'text': 'GR_RECONSTRUCTED', 'font': {'color': 'black', 'size': font_size}},
        'tickfont': {'color': 'black', 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .89, 'overlaying': 'x1',
        'title_standoff': .1, 'dtick': 40, 'range': [0, 200], 'type': 'linear', 'zeroline': False
    },
    'RHOB_RECONSTRUCTED': {
        'title': {'text': 'RHOB_RECONSTRUCTED', 'font': {'color': 'black', 'size': font_size}},
        'tickfont': {'color': 'black', 'size': font_size}, 'zeroline': False,
        'side': 'top', 'anchor': 'free', 'position': .89, 'overlaying': 'x3',
        'tickformat': ".2f", 'tick0': 1.95, 'dtick': 0.2, 'tickangle': -90,
        'range': [1.95, 2.95], 'type': 'linear'
    },
    'NPHI_RECONSTRUCTED': {
        'title': {'text': 'NPHI_RECONSTRUCTED', 'font': {'color': 'black', 'size': font_size}},
        'tickfont': {'color': 'black', 'size': font_size}, 'zeroline': False,
        'tickformat': ".2f", 'tick0': -.15, 'dtick': 0.12, 'range': [.45, -.15], 'type': 'linear', 'tickangle': -90,
        'side': 'top', 'anchor': 'free', 'position': .89, 'overlaying': 'x4',
    },
    'PEF_RECONSTRUCTED': {
        'title': {'text': 'PEF_RECONSTRUCTED', 'font': {'color': 'black', 'size': font_size}},
        'tickfont': {'color': 'black', 'size': font_size}, 'zeroline': False,
        'side': 'top', 'anchor': 'free', 'position': .89, 'overlaying': 'x5',
        'dtick': 1, 'range': [0, 7], 'type': 'linear'
    },
    'DTC_RECONSTRUCTED': {
        'title': {'text': 'DTC_RECONSTRUCTED', 'font': {'color': 'black', 'size': font_size}},
        'tickfont': {'color': 'black', 'size': font_size}, 'zeroline': False,
        'side': 'top', 'anchor': 'free', 'position': .89, 'overlaying': 'x6',
        'dtick': 20, 'range': [30, 150], 'type': 'linear'
    },
    'AVG_ERROR': {
        'title': {'text': 'AVG_ERROR', 'font': {'color': 'black', 'size': font_size}},
        'tickfont': {'color': 'black', 'size': font_size}, 'zeroline': False,
        'side': 'top', 'anchor': 'free', 'position': .89, 'type': 'linear'
    },
    'PHIT': {
        'title': {'text': 'PHIT', 'font': {'color': COLOR_DICT['PHIT'], 'size': font_size}},
        'tickfont': {'color': COLOR_DICT['PHIT'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .85, 'title_standoff': .1,
        'dtick': 0.1, 'range': [0, 0.5], 'type': 'linear', 'zeroline': False
    },
    'PHIT_CONSTRUCTED': {
        'title': {'text': 'PHIT_CONSTRUCTED', 'font': {'color': 'black', 'size': font_size}},
        'tickfont': {'color': 'black', 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .89, 'title_standoff': .1,
        'dtick': 0.1, 'range': [0, 0.5], 'type': 'linear', 'zeroline': False, 'overlaying': 'x7',
    },
    'CALI': {
        'title': {'text': 'CALI', 'font': {'color': COLOR_DICT['CALI'], 'size': font_size}},
        'tickfont': {'color': COLOR_DICT['CALI'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .89, 'title_standoff': .1, 'overlaying': 'x1',
        'dtick': 6, 'range': [6, 24], 'type': 'linear', 'showgrid': False
    },
}

# Clastic Data Example

In [ ]:
with open(r'data\01_raw\36_7-3.las', 'rb') as f:
    df, header = las.read_las_files([f])
df['RT'] = df['RDEP']

# Mask outside threshold
df = handle_outer_limit(df, True)

# Flag bad hole
df = badhole_flagging(df) if 'CALI' in df.columns else df
clear_output()

In [ ]:
import numpy as np

neu_den_df = df[['DEPTH', 'GR', 'NPHI', 'RHOB', 'DTC', 'PEF']].dropna()
# neu_den_df[['PEF', 'DTC']] = np.nan
minerals = ['QUARTZ', 'SHALE']
df_ssc_model = MultiMineral(minerals).estimate_lithology(
    gr=neu_den_df['GR'], nphi=neu_den_df['NPHI'], rhob=neu_den_df['RHOB'],
    pef=neu_den_df['PEF'], dtc=neu_den_df['DTC']
)
df_ssc_model['DEPTH'] = neu_den_df.DEPTH

df_ssc = df.merge(df_ssc_model, how='left', on='DEPTH', suffixes=('_ORI', ''))

In [ ]:
fig = plotly_log(df_ssc, 'meter', trace_defs=TRACE_DEFS, xaxis_defs=XAXIS_DEFS)
fig.show(config=dict(scrollZoom=True))
# fig.write_html('plot_no pef log curves.html')

In [ ]:
neu_den_df = df[['DEPTH', 'GR', 'NPHI', 'RHOB', 'DTC', 'PEF']].dropna()
df_ssc_model = MultiMineral().estimate_lithology(
    gr=neu_den_df['GR'], nphi=neu_den_df['NPHI'], rhob=neu_den_df['RHOB'],
    pef=neu_den_df['PEF'], dtc=neu_den_df['DTC']
)
df_ssc_model['DEPTH'] = neu_den_df.DEPTH

df_ssc = df.merge(df_ssc_model, how='left', on='DEPTH', suffixes=('_ORI', ''))

In [ ]:
fig = plotly_log(df_ssc, 'meter', trace_defs=TRACE_DEFS, xaxis_defs=XAXIS_DEFS)
fig.show(config=dict(scrollZoom=True))
# fig.write_html('plot_all curves.html')

In [ ]:
import numpy as np

neu_den_df = df[['DEPTH', 'GR', 'NPHI', 'RHOB', 'DTC', 'PEF']].dropna()
neu_den_df[['PEF']] = np.nan
df_ssc_model = MultiMineral(minerals).estimate_lithology(
    gr=neu_den_df['GR'], nphi=neu_den_df['NPHI'], rhob=neu_den_df['RHOB'],
    pef=neu_den_df['PEF'], dtc=neu_den_df['DTC']
)
df_ssc_model['DEPTH'] = neu_den_df.DEPTH

df_ssc = df.merge(df_ssc_model, how='left', on='DEPTH', suffixes=('_ORI', ''))

In [ ]:
fig = plotly_log(df_ssc, 'meter', trace_defs=TRACE_DEFS, xaxis_defs=XAXIS_DEFS)
fig.show(config=dict(scrollZoom=True))
# fig.write_html('plot_basic log curves.html')

In [ ]:
import numpy as np

neu_den_df = df[['DEPTH', 'GR', 'NPHI', 'RHOB', 'DTC', 'PEF']].dropna()
neu_den_df[['DTC']] = np.nan
df_ssc_model = MultiMineral(minerals).estimate_lithology(
    gr=neu_den_df['GR'], nphi=neu_den_df['NPHI'], rhob=neu_den_df['RHOB'],
    pef=neu_den_df['PEF'], dtc=neu_den_df['DTC']
)
df_ssc_model['DEPTH'] = neu_den_df.DEPTH

df_ssc = df.merge(df_ssc_model, how='left', on='DEPTH', suffixes=('_ORI', ''))

In [ ]:
fig = plotly_log(df_ssc, 'meter', trace_defs=TRACE_DEFS, xaxis_defs=XAXIS_DEFS)
fig.show(config=dict(scrollZoom=True))
# fig.write_html('plot_no dtc log curves.html')

# Carbonate Data Example (COSTA)

In [ ]:
with open(r'data\01_raw\COSTA\HW-10.las', 'rb') as f:
    df_c, header = las.read_las_files([f])

# Mask outside threshold
df_c = handle_outer_limit(df_c, True)

# Flag bad hole
df_c = badhole_flagging(df_c) if 'CALI' in df_c.columns else df_c
clear_output()

In [ ]:
import numpy as np

neu_den_df = df_c[['DEPTH', 'GR', 'NPHI', 'RHOB']].dropna()
neu_den_df[['DTC', 'PEF']] = np.nan
minerals_c = ['CALCITE', 'SHALE']
df_c_model = MultiMineral(minerals=minerals_c).estimate_lithology(
    gr=neu_den_df['GR'], nphi=neu_den_df['NPHI'], rhob=neu_den_df['RHOB'],
    pef=neu_den_df['PEF'], dtc=neu_den_df['DTC']
)
df_c_model['DEPTH'] = neu_den_df.DEPTH

df_carb = df_c.merge(df_c_model, how='left', on='DEPTH', suffixes=('_ORI', ''))

In [ ]:
fig = plotly_log(df_carb, 'meter', trace_defs=TRACE_DEFS, xaxis_defs=XAXIS_DEFS)
fig.show(config=dict(scrollZoom=True))
# fig.write_html('plot_no dtc pef carbonate log curves.html')

In [ ]:
import numpy as np

neu_den_df = df_c[['DEPTH', 'GR', 'NPHI', 'RHOB']].dropna()
neu_den_df[['DTC', 'PEF']] = np.nan
df_c_model = MultiMineral().estimate_lithology(
    gr=neu_den_df['GR'], nphi=neu_den_df['NPHI'], rhob=neu_den_df['RHOB'],
    pef=neu_den_df['PEF'], dtc=neu_den_df['DTC']
)
df_c_model['DEPTH'] = neu_den_df.DEPTH

df_carb = df_c.merge(df_c_model, how='left', on='DEPTH', suffixes=('_ORI', ''))

In [ ]:
fig = plotly_log(df_carb, 'meter', trace_defs=TRACE_DEFS, xaxis_defs=XAXIS_DEFS)
fig.show(config=dict(scrollZoom=True))
# fig.write_html('plot_no dtc pef carbonate all mineral log curves.html')

In [ ]:
import numpy as np

neu_den_df = df_c[['DEPTH', 'GR', 'NPHI', 'RHOB']].dropna()
neu_den_df[['DTC', 'PEF']] = np.nan
minerals = ['CALCITE', 'DOLOMITE', 'SHALE']
df_c_model = MultiMineral(minerals=minerals).estimate_lithology(
    gr=neu_den_df['GR'], nphi=neu_den_df['NPHI'], rhob=neu_den_df['RHOB'],
    pef=neu_den_df['PEF'], dtc=neu_den_df['DTC']
)
df_c_model['DEPTH'] = neu_den_df.DEPTH

df_carb = df_c.merge(df_c_model, how='left', on='DEPTH', suffixes=('_ORI', ''))

In [ ]:
fig = plotly_log(df_carb, 'meter', trace_defs=TRACE_DEFS, xaxis_defs=XAXIS_DEFS)
fig.show(config=dict(scrollZoom=True))
# fig.write_html('plot_no dtc pef carbonate all mineral log curves.html')